In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [84]:
house = pd.read_csv("my_clean_data.csv")
house.head()

,bhk,total_sqft,bath,price,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Phase JP Nagar,6th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,4,2850.0,4.0,428.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1630.0,3.0,194.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1200.0,6.0,125.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1875.0,2.0,235.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7,930.0,4.0,85.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
house.shape

(6842, 241)

# model building

In [86]:
target_y = house['price']
features_x = house.drop(['price'] , axis=1)
features_x.shape , target_y.shape 

((6842, 240), (6842,))

In [87]:
from sklearn.model_selection import train_test_split

train_x,test_x , train_y,test_y = train_test_split(features_x , target_y , test_size=0.2 , random_state=10)
train_x.shape , test_x.shape

((5473, 240), (1369, 240))

In [88]:
from sklearn.linear_model import LinearRegression  ,  Lasso , Ridge
from sklearn.metrics import mean_absolute_error

In [89]:
# lr = LinearRegression()
# lr.fit(train_x , train_y)
# score = lr.score(test_x , test_y) 
# print("score : " ,  score)
# y_pred = lr.predict(test_x)
# error = mean_absolute_error(y_pred , test_y)
# print("error : " , error)

In [90]:
# from sklearn.preprocessing import PolynomialFeatures
# from sklearn.pipeline import make_pipeline

In [91]:
# degree = 0
# lr = make_pipeline(PolynomialFeatures(degree) ,  LinearRegression() )
# lr.fit(train_x2 , train_y2)
# score = lr.score(test_x2 , test_y2) 
# print("score : " ,  score)
# y_pred2 = lr.predict(test_x2)
# error = mean_absolute_error(y_pred2 , test_y2)
# print("error : " , error)

In [92]:
from sklearn.model_selection import ShuffleSplit , cross_val_score

In [93]:
cv = ShuffleSplit(n_splits=4  , test_size=0.2 ,random_state=0)
cross_val_score(LinearRegression() , features_x , target_y ,cv=cv)

array([0.79164179, 0.81036219, 0.8087572 , 0.69557672])

In [94]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import  DecisionTreeRegressor

In [95]:
def find_best_model_using_GridSearcCv(x , y ):

    algos = {
        'linear_regression' : {
            'model' : LinearRegression() ,
            'params'  : {
                 'normalize': [True , False]
            }
        },
        # 'ridge_regression' : {
        #     'model' : Ridge() ,
        #     'params'  : {
        #           'alpha' : [1,2] ,
        #     }
        # },
        'lasso_regression' : {
            'model' : Lasso() ,
            'params'  : {
                 'alpha' : [1,2] ,
                'selection': ['random' , 'cyclic']
            }
        },
        'decision_tree' : {
            'model' : DecisionTreeRegressor() ,
            'params'  : {
                 'criterion' : ['mse' , 'friedman_mse'] ,
                'splitter': ['best' , 'random']
            }
        },
    }

    scores = pd.DataFrame([] , columns = ['model' , 'best_score' , 'best_params'])

    cv = ShuffleSplit(n_splits=5 , random_state= 0 , test_size=0.2)
    
    for algo_name , config in algos.items():
        gs = GridSearchCV( config['model'] , config['params'] , cv = cv , return_train_score=False)
        
        gs.fit(x , y)

        scores = scores.append( {
            'model' : algo_name , 
            'best_score' : gs.best_score_ ,
            'best_params' : gs.best_params_ ,
        } , ignore_index=True)
    
    return scores

# scores_df = find_best_model_using_GridSearcCv(features_x , target_y)
# scores_df.to_csv('best_model.csv')
# scores_df

In [97]:
LinearRegression().get_params().values()

dict_values([True, True, None, False, False])

In [98]:
lr_model = LinearRegression(normalize = False)
lr_model.fit(train_x , train_y)

score = lr_model.score(test_x , test_y) 

print("score : " ,  score)

score :  0.8055448831825935


In [99]:
pred_y = lr_model.predict(test_x )

error = mean_absolute_error(pred_y , test_y)
print("error : " , error)


error :  13.850560476764821


In [100]:
def predict_price( bhk  ,sqft , bath ,location  ):

    try:
        location_index = np.where(features_x.columns == location)[0][0] #(array([5], dtype=int64), )
    except:
        print(location  +  " not found !!!!!!!")
        return 0

    house_data  = np.zeros( features_x.shape[1] )

    house_data[0] = bhk 
    house_data[1] = sqft
    house_data[2] = bath

    house_data[location_index] = 1

    # print(house_data)
    price = lr_model.predict( [house_data] )[0]
    return price

price  = predict_price(4 ,2850.0 , 4.0  ,  '1st Block Jayanagar'  )
# price  = predict_price(2 ,1630 , 3 	 ,  '1st Block Jayanagar'  )
print("price : %.2f "  % price  , " Rs")

price : 312.44   Rs


In [101]:
features_x.head(2)

,bhk,total_sqft,bath,1st Block Jayanagar,1st Phase JP Nagar,2nd Phase Judicial Layout,2nd Stage Nagarbhavi,5th Phase JP Nagar,6th Phase JP Nagar,7th Phase JP Nagar,...,Vijayanagar,Vishveshwarya Layout,Vishwapriya Layout,Vittasandra,Whitefield,Yelachenahalli,Yelahanka,Yelahanka New Town,Yelenahalli,Yeshwanthpur
0,4,2850.0,4.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1630.0,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [102]:
target_y.head(2)

0    428.0
1    194.0
Name: price, dtype: float64

# saving the model

In [103]:
import pickle
import json 


In [104]:
with open('benglore_house_prices_model.pickle' , 'wb') as f:
    pickle.dump(lr_model , f)

In [105]:
columns = {'data_columns' : [col.lower() for col in features_x.columns]}

with open('columns.json' , 'w') as f:
    f.write(json.dumps(columns))